In [ ]:
# default_exp distillation

# Knowledge Distillation
> Classes for performing task-specific distillation

The classes in this module are adapted from the following resources:

* The scripts from the [distillation example](https://github.com/huggingface/transformers/tree/master/examples/research_projects/distillation) in the `transformers` repository

#hide
## Load libraries

In [ ]:
#export
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformerlab.question_answering import *

## Trainer

In [ ]:
#export
class DistillationTrainingArguments(QuestionAnsweringTrainingArguments):
    def __init__(self, *args, alpha_ce=0.5, alpha_distil=0.5, temperature=2.0, **kwargs):
        super().__init__(*args, **kwargs)
        
        self.alpha_ce = alpha_ce
        self.alpha_distil = alpha_distil
        self.temperature = temperature

In [ ]:
#export
class DistillationTrainer(QuestionAnsweringTrainer):
    def __init__(self, *args, teacher_model=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.teacher = teacher_model
        self.teacher.eval()
        self.train_dataset.set_format(
            type=self.train_dataset.format["type"], columns=list(self.train_dataset.features.keys()))
        
    def compute_loss(self, model, inputs):
        inputs_stu = {
            "input_ids": inputs['input_ids'],
            "attention_mask": inputs['attention_mask'],
            "start_positions": inputs['start_positions'],
            "end_positions": inputs['end_positions'],
            }
        outputs_stu = model(**inputs_stu)
        loss = outputs_stu.loss
        start_logits_stu = outputs_stu.start_logits
        end_logits_stu = outputs_stu.end_logits
        
        with torch.no_grad():
            outputs_tea = self.teacher(
                input_ids=inputs["input_ids"], 
                token_type_ids=inputs["token_type_ids"],
                attention_mask=inputs["attention_mask"])
            start_logits_tea = outputs_tea.start_logits
            end_logits_tea = outputs_tea.end_logits
        assert start_logits_tea.size() == start_logits_stu.size()
        assert end_logits_tea.size() == end_logits_stu.size()
        
        loss_fct = nn.KLDivLoss(reduction="batchmean")
        loss_start = (loss_fct(
            F.log_softmax(start_logits_stu / self.args.temperature, dim=-1),
            F.softmax(start_logits_tea / self.args.temperature, dim=-1)) * (self.args.temperature ** 2))
        loss_end = (loss_fct(
            F.log_softmax(end_logits_stu / self.args.temperature, dim=-1),
            F.softmax(end_logits_tea / self.args.temperature, dim=-1)) * (self.args.temperature ** 2))
        loss_logits = (loss_start + loss_end) / 2.0
        loss = self.args.alpha_distil * loss_logits + self.args.alpha_distil * loss
        return loss